In [25]:
import numpy as np 
import pandas as pd 
import tweepy

In [26]:
df_user_details=pd.read_csv('user_details.csv') #More detailed information about a user (friend count, followers count)  

In [27]:
df_followers=pd.read_csv('followers.csv')  #The follower network as Twitter ids

In [28]:
df_skip=pd.read_csv('skip.csv')   # A master record of which ids have had their followers retrieved

In [29]:
df_user_details.head()

,name,screen_name,created_at,id,friends_count,followers_count
0,The Academy,TheAcademy,2010-10-08 16:09:08,200163448,2185,3663377
1,Sergio P.Burguera,PozaSergio,2020-12-27 19:12:36,1343273549428097027,512,20
2,Carmen Ava Maria,CarmenAvaMaria1,2018-11-05 12:20:13,1059420115094200320,1325,458
3,VoiceBurma_21,VoiceBurma,2021-04-26 08:38:17,1386600424065536001,19,0
4,K Suresh,vzsuresh,2017-02-28 05:50:56,836453613207592961,96,17


In [30]:
df_user_details.shape

(13, 6)

In [31]:
df_followers.head()

,id,followers
0,200163448,1343273549428097027
1,200163448,1059420115094200320
2,200163448,1386600424065536001
3,200163448,836453613207592961
4,200163448,1244542608841400323


In [32]:
df_followers.shape

(5621, 2)

In [33]:
# df_followers2=df_followers.loc[3000:,:]

In [34]:
df_skip.head()

,id
0,200163448
1,1343273549428097027
2,1059420115094200320
3,1386600424065536001
4,836453613207592961


In [35]:
df_skip.shape

(14, 1)

In [36]:

df_followers.rename(columns = {'id':'source', 'followers':'target',
                              }, inplace = True)

In [37]:
# API Authentication

CONSUMER_KEY = 'hheqJPV07LZzIBCVGFEUt2ojI'
CONSUMER_SECRET = 'X3x4ausVn0KYLmclg6pAnPp4esnJ532J6z2Sla3QB1yAr37VKl'
ACCESS_TOKEN = '1065450120433553410-6H5N0JJYAdji06KNE3ozlgJj1zgeOL'
ACCESS_TOKEN_SECRET = '5kCiBBcDGOfwkmORFHD4cVVaNOAujUu29ML087KXsLlTn'

In [38]:

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


#### alternatively - using the users ID 

In [ ]:
user_list = ["200163448"] 
follower_list = []
for user in user_list:
    followers = []
    try:
        for page in tweepy.Cursor(api.followers_ids, user_id=user).pages():
            followers.extend(page)
            print(len(followers))
    except tweepy.TweepError:
        print("error")
        continue
    follower_list.append(followers)

In [ ]:
df = pd.DataFrame(columns=['source','target']) #Empty DataFrame
df['target'] = follower_list[0] #Set the list of followers as the target column
df['source'] = 1324592748 #Set my user ID as the source 

In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(df, 'source', 'target') #Turn df into graph
pos = nx.spring_layout(G) #specify layout for visual

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(30, 10))
plt.style.use('ggplot')
nodes = nx.draw_networkx_nodes(G, pos,
                               alpha=0.8)
nodes.set_edgecolor('k')
nx.draw_networkx_labels(G, pos, font_size=8)
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.2)

In [ ]:
ser_list = list(df_followers['target']) #Use the list of followers we extracted in the code above i.e. my 450 followers
for userID in user_list:
    print(userID)
    followers = []
    follower_list = []

    # fetching the user
    user = api.get_user(userID)

    # fetching the followers_count
    followers_count = user.followers_count

    try:
        for page in tweepy.Cursor(api.followers_ids, user_id=userID).pages():
            followers.extend(page)
            print(len(followers))
            if followers_count >= 5000: #Only take first 5000 followers
                break
    except tweepy.TweepError:
        print("error")
        continue
    follower_list.append(followers)
    temp = pd.DataFrame(columns=['source', 'target'])
    temp['target'] = follower_list[0]
    temp['source'] = userID
    df = df.append(temp)
    df.to_csv("networkOfFollowers.csv")

In [ ]:
username = 'TheAcademy'
count = 150
try:     
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)
 
 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df = pd.DataFrame(tweets_list)
except BaseException as e:
      print('failed on_status,',str(e))
      time.sleep(3)

In [ ]:
tweets_df.head()

In [ ]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([RT])', ' ', str(tweet).lower()).split())


In [ ]:
# Funciton to analyze Sentiment

from textblob import TextBlob
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [ ]:

def prepCloud(Topic_text,Topic):
    Topic = str(Topic).lower()
    Topic=' '.join(re.sub('([^0-9A-Za-z \t])', ' ', Topic).split())
    Topic = re.split("\s+",str(Topic))
    stopwords = set(STOPWORDS)
    stopwords.update(Topic) ### Add our topic in Stopwords, so it doesnt appear in wordClous
    ###
    text_new = " ".join([txt for txt in Topic_text.split() if txt not in stopwords])
    return text_new

In [ ]:
# Call function to get Clean tweets

# tweets_df['clean'] = tweets_df['2'].apply(lambda x : clean_tweet(x))
# tweets_df.head(5)


tweets_df.rename(columns = {1:'Id', 2:'tweetraw'}, inplace = True)

In [ ]:
#Call function to get Clean tweets

tweets_df['CleanTweet'] = tweets_df['tweetraw'].apply(lambda x : clean_tweet(x))
tweets_df.head(5)

In [ ]:
tweets_df["Sentiment"] = tweets_df["CleanTweet"].apply(lambda x : analyze_sentiment(x))
tweets_df.head(5)

In [ ]:
tweets_df["Sentiment"].value_counts()